This code is responsible for calculating, for each provider, the number of tokens from each prompt.

In [ ]:
# Loading the ENV file for API KEYS
from dotenv import load_dotenv
import os
load_dotenv()

API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
import os

def getPrompts(promptsPath="./prompts/"):
    promptsFiles = os.listdir(promptsPath)
    prompts = []

    for filename in promptsFiles:
        if filename.endswith(".txt"):
            file_path = os.path.join(promptsPath, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                prompts.append(content)

    promptsFiles = [x.replace('.txt','') for x in promptsFiles]
    return promptsFiles, prompts

files, prompts = getPrompts()

In [ ]:
# OpenAI

import tiktoken

models = ['gpt-5','gpt-5-mini']

for model in models:
    for index, prompt in enumerate(prompts):
        encoding = tiktoken.encoding_for_model(model)
        print(f"Model: {model}, Prompt: {files[index]} Size: {len(encoding.encode(prompt))} tokens")


In [ ]:
from anthropic import Anthropic

client = Anthropic(api_key=ANTHROPIC_API_KEY)


models = ['claude-sonnet-4-5', 'claude-haiku-4-5']

for model in models:
    for index, prompt in enumerate(prompts):
        response = client.messages.count_tokens(
        system=prompt,
        model=model,
        messages=[
            {"role": "user", "content": "Test"}
        ]
        )
        print(f'{files[index]},{response.input_tokens}')
    

In [ ]:
# Google - GEMINI
from google import genai
from google.genai import types
results = {}

models = ['gemini-2.5-pro','gemini-2.5-flash']


client = genai.Client(api_key=API_KEY)
for model in models:
    results[model] = {}
    for index, prompt in enumerate(prompts):

        response = client.models.count_tokens(
        model=model,
        contents=prompt
        )
        
        print(files[index])
        print(response.total_tokens)
